In [5]:
import os
import json
import glob
from PIL import Image

classes = ['star', 'arrow backward', 'arrow forward', 'more', 'menu', 'search', 'close', 'add', 
           'expand more', 'play', 'check', 'share', 'chat', 'settings', 'info', 'home', 'refresh', 
           'time', 'emoji', 'edit', 'notifications', 'call', 'pause', 'send', 'delete', 'video cam']  

def convert_to_yolo_format(box, img_width, img_height):
    x_center = (box[0] + box[2]) / 2 / img_width
    y_center = (box[1] + box[3]) / 2 / img_height
    width = (box[2] - box[0]) / img_width
    height = (box[3] - box[1]) / img_height
    return [x_center, y_center, width, height]

def process_json_files(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    json_files = glob.glob(os.path.join(input_dir, '*.json'))
    for json_file in json_files:
        with open(json_file) as f:
            data = json.load(f)
        
        
        if 'size' not in data or not data['size']:
            basename_no_ext = os.path.splitext(os.path.basename(json_file))[0]
            output_file = os.path.join(output_dir, basename_no_ext + '.txt')
            open(output_file, 'w').close()  
            continue
        
        img_width = data['size']['width']
        img_height = data['size']['height']

        
        if img_width == 0 or img_height == 0:
            basename_no_ext = os.path.splitext(os.path.basename(json_file))[0]
            output_file = os.path.join(output_dir, basename_no_ext + '.txt')
            open(output_file, 'w').close()  
            continue
        
        
        if 'outputs' not in data or 'object' not in data['outputs'] or not data['outputs']['object']:
            basename_no_ext = os.path.splitext(os.path.basename(json_file))[0]
            output_file = os.path.join(output_dir, basename_no_ext + '.txt')
            open(output_file, 'w').close()  
            continue

        annotations = data['outputs']['object']
        yolo_annotations = []
        for ann in annotations:
            class_name = ann['name'].lower()
            if class_name not in classes:
                continue
            class_id = classes.index(class_name)
            box = [ann['bndbox']['xmin'], ann['bndbox']['ymin'], ann['bndbox']['xmax'], ann['bndbox']['ymax']]
            yolo_box = convert_to_yolo_format(box, img_width, img_height)
            yolo_annotations.append([class_id] + yolo_box)
        
        
        basename_no_ext = os.path.splitext(os.path.basename(json_file))[0]
        output_file = os.path.join(output_dir, basename_no_ext + '.txt')
        with open(output_file, 'w') as f:
            for ann in yolo_annotations:
                f.write(' '.join(map(str, ann)) + '\n')


input_dir = r"F:\prism\Final_Prism_Dataset\json"  
output_dir = r"F:\prism\Final_Prism_Dataset\text"  

process_json_files(input_dir, output_dir)
